In [1]:
import pandas as pd  #importing all the important packages
import numpy as np
import matplotlib.pyplot as plt #currently not used
import seaborn as sns #currently not used
import pickle
pickle_in = open('gen5_pokemon.pickle','rb')
pokemon = pickle.load(pickle_in)
pickle_in = open('all_gen5_moves.pickle','rb')
moves = pickle.load(pickle_in)

# Damage multiplier for type matchups

The function accepts two types, the first type is the attacking move type and the second is the defenders type (if the defender has multiple types this function should be run multiple times and multiplied together).

In [2]:
def type_weakness(type1,type2):
    if type1 == 'normal':
        if type2 in ['rock' ,'steel']:
            return 0.5
        elif type2 == 'ghost':
            return 0
    if type1 == 'fire':
        if type2 in ['water','fire','rock', 'dragon']:
            return 0.5
        elif type2 in ['grass', 'ice', 'bug','steel']:
            return 2
    if type1 == 'water':
        if type2 in ['water', 'grass', 'dragon']:
            return 0.5
        if type2 in ['fire', 'ground', 'rock']:
            return 2
    if type1 == 'grass':
        if type2 in ['fire', 'grass', 'poison' , 'dragon', 'steel', 'bug', 'flying']:
            return 0.5
        if type2 in ['water' , 'ground' ,'rock']:
            return 2
    if type1 == 'electric':
        if type2 == 'ground':
            return 0
        if type2 in ['grass', 'electric', 'dragon']:
            return 0.5
        if type2 in ['water','flying']:
            return 2
    if type1 == 'ice':
        if type2 in ['fire', 'water', 'ice', 'steel']:
            return 0.5
        if type2 in ['grass','flying','ground', 'dragon']:
            return 2
    if type1 == 'fighting':
        if type2 == 'ghost':
            return 0
        if type2 in ['poison', 'flying', 'psychic', 'bug', 'fairy']:
            return 0.5
        if type2 in ['normal', 'ice','rock', 'dark', 'steel']:
            return 2
    if type1 == 'poison':
        if type2 == 'steel':
            return 0
        if type2 in ['poison', 'ground', 'rock', 'ghost']:
            return 0.5
        if type2 in['fairy', 'grass']:
            return 2
    if type1 == 'ground':
        if type2 == 'flying':
            return 0
        if type2 in['grass','bug']:
            return 0.5
        if type2 in ['fire', 'electric','rock', 'steel', 'poison']:
            return 2
    if type1 == 'flying':
        if type2 in['electric','steel', 'rock']:
            return 0.5
        if type2 in ['grass', 'fighting', 'bug']:
            return 2
    if type1 == 'psychic':
        if type2 == 'dark':
            return 0
        if type2 in['steel','psychic']:
            return 0.5
        if type2 in ['fighting', 'poison']:
            return 2
    if type1 == 'bug':
        if type2 in ['fire', 'fighting', 'poison', 'flying', 'ghost' ,'steel', 'fairy']:
            return 0.5
        if type2 in ['dark','psychic','ghost']:
            return 2
    if type1 == 'rock':
        if type2 in ['steel', 'fighting', 'ground']:
            return 0.5
        if type2 in ['bug' ,'flying','ice','fire']:
            return 2
    if type1 == 'ghost':
        if type2 == 'normal':
            return 0
        if type2 == 'dark':
            return 0.5
        if type2 in ['psychic','ghost']:
            return 2
    if type1 == 'dragon':
        if type2 == 'fairy':
            return 0
        if type2 == 'steel':
            return 0.5
        if type2 == 'dragon':
            return 2
    if type1 == 'dark':
        if type2 in ['fighting','dark','fairy']:
            return 0.5
        if type2 in ['ghost','psychic']:
            return 2
    if type1 == 'steel':
        if type2 in ['steel','fire','water','electric']:
            return 0.5
        if type2 in ['ice','rock','fairy']:
            return 2
    if type1 == 'fairy':
        if type2 in ['fire', 'poison', 'steel']:
            return 0.5
        if type2 in ['dragon', 'fighting', 'dark']:
            return 2
    return 1

# Damage Calculations

In [3]:
def dmg(ratio,lvl,crit,power,movetype,stab,deftype1,deftype2,other):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*ratio
    #extras
    dmg =dmg/50+2
    #burned (and not guts)
    #if burn == True:
    #    dmg = dmg/2
    #screen
    #if screen == True:
    #    if double == True:
    #        dmg = dmg*2/3
    #    else:
    #        dmg = dmg/2
    #in double battle dmg split between targets
    #if double == True and targetall == False:
    #    dmg = dmg/2
    #flash fire
    #if FF == True:
    #    dmg = dmg*1.5
    #weather
    #if weather == True:
    #    dmg = dmg*1.5
    #item multiplier (assuming this goes here)
    #dmg = dmg*item
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)
        dmg = dmg*type_weakness(movetype,deftype2)
    #if crits
    if crit == True:
        dmg = 1.5*dmg
    #weatherball revenge (only in clear weather)
    #if WBR == True:
    #    dmg = dmg*2
    #helping hand
    #if HH == True:
    #    dmg = dmg*1.5
    #electric type charge effect
    #if charge == True:
    #    dmg =dmg*2
    #any other effects; e.g. abilities
    dmg = dmg*other
    return [int(dmg*(85/100)),int(dmg)]

# Base Stats at Level

We use the following function to determine a pokemons base stats at a certain level, this does not include IVs, EVs or natures. However, in the function IVs and EVs can be seen. Natures would multiply one stat by 1.1 and another by 0.9.

In [4]:
def base_stats_at_lvl(Pokemon,level,df):
    pkm = df[df.index == Pokemon]
    IV = 0#[0,0,0,0,0,0]
    EV = 0#[0,0,0,0,0,0]
    HP = ((2*pkm['HP']+IV+(EV/4))*level)/100+level+10
    ATK = ((2*pkm['ATK']+IV+(EV/4))*level)/100+5
    DEF = ((2*pkm['DEF']+IV+(EV/4))*level)/100+5
    SPATK = ((2*pkm['SPATK']+IV+(EV/4))*level)/100+5
    SPDEF = ((2*pkm['SPDEF']+IV+(EV/4))*level)/100+5
    SPD = ((2*pkm['SPD']+IV+(EV/4))*level)/100+5
    return [int(HP),int(ATK),int(DEF),int(SPATK),int(SPDEF),int(SPD)]

# A/D Ratio

We create a function to calculate the A/D ratio between two pokemon. This is the difference in the attack stat with the defence stat or the special attack stat with the special defence stat. (There are some exceptions such as Psyshock and Psystrike).

In [5]:
def phys_spec_ratio(atk_pkm,atk_lvl,def_pkm,def_lvl,df):
    pkm1 = base_stats_at_lvl(atk_pkm,atk_lvl,df)
    pkm2 = base_stats_at_lvl(def_pkm,def_lvl,df)
    phys = pkm1[1]/pkm2[2]
    spec = pkm1[3]/pkm2[4]
    return[float(phys),float(spec)]

# Speed Test

We create a function to test the speed comparison between two pokemon.

In [6]:
def speed_test(atk_pkm,atk_lvl,def_pkm,def_lvl,df):
    pkm1 = base_stats_at_lvl(atk_pkm,atk_lvl,df)
    pkm2 = base_stats_at_lvl(def_pkm,def_lvl,df)
    spd1 = pkm1[5]
    spd2 = pkm2[5]
    spd1_posNat = spd1*1.1
    spd1_negNat = spd1*0.9
    print(atk_pkm +' negative nature: ' + str(spd1_negNat))
    print(atk_pkm +' neutral nature: ' + str(spd1))
    print(atk_pkm +' positive nature: ' + str(spd1_posNat))
    print(def_pkm +' base speed: ' + str(spd2))
    #return[float(spd1_negNat),float(spd1),float(posNat),float(spd2)]
#speed_test('Pignite',20,'Watchog',20,pokemon)

# Get Moves

Returns the dataframe of moves associated with a pokemon.

In [7]:
def get_moves(pokemon,moves):
    return moves[moves['POKEMON']==pokemon]

For moves which the power is based on weight, e.g. heat crash, grass knot, heavy slam and low kick damage is calculated thus.

In [8]:
def both_weight(p1,p2,pokemon):
    w1 = pokemon[pokemon.index == p1].WEIGHTKGS.values.astype(float)
    w2 = pokemon[pokemon.index == p2].WEIGHTKGS.values.astype(float)
    if w2/w1*100 <= 20:
        return 120
    elif w2/w1*100 <= 25:
        return 100
    elif w2/w1*100 <= 100/3:
        return 80
    elif w2/w1*100 <= 50:
        return 60
    else:
        return 40

In [9]:
def target_weight(p2,pokemon):
    w = pokemon[pokemon.index == p2].WEIGHTKGS.values.astype(float)
    if w < 10:
        return 20
    elif w < 25:
        return 40
    elif w < 50:
        return 60
    elif w < 100:
        return 80
    elif w < 200:
        return 100
    else:
        return 120

# BATTLE

We take the example of two pokemon with no IVs,EVs, or natures attacking one another. The function takes in the two pokemon and the move being used as well as their levels

In [10]:
def battle(p1_name,p1_lvl,p2_name,p2_lvl,move,crit,pokemon,moves):
    a_d = phys_spec_ratio(p1_name,p1_lvl,p2_name,p2_lvl,pokemon)
    moves = get_moves(p1_name,moves)
    move = moves[moves.index == move]
    if move.CATEGORY.values == 'physical':
        ratio = a_d[0]
    elif move.CATEGORY.values == 'special':
        ratio = a_d[1]
    else:
        ratio = 0
    p1 = pokemon[pokemon.index == p1_name]
    p2 = pokemon[pokemon.index == p2_name]
    STAB = (move.TYPE.values == p1.TYPE1.values) or (move.TYPE.values == p1.TYPE2.values)
    other = 1
    if (move.index.values == 'Heat Crash') or (move.index.values == 'Heavy Slam'):
        power = both_weight(p1_name,p2_name,pokemon)
        damage = dmg(ratio, p1_lvl, crit, power, move['TYPE'].values,STAB,p2['TYPE1'].values,p2['TYPE2'].values,other)
    elif (move.index.values == 'Grass Knot') or (move.index.values == 'Low Kick'):
        power = target_weight(p2_name,pokemon)
        damage = dmg(ratio, p1_lvl, crit, power, move['TYPE'].values,STAB,p2['TYPE1'].values,p2['TYPE2'].values,other)
    else:
        damage = dmg(ratio, p1_lvl, crit, move['POWER'].values.astype(str).astype(int), move['TYPE'].values,STAB,p2['TYPE1'].values,p2['TYPE2'].values,other)
    p2_hp = base_stats_at_lvl(p2_name,p2_lvl,pokemon)[0]
    return [int(damage[0]/p2_hp*100),int(damage[1]/p2_hp*100)]
#battle('Emboar',30,'Lillipup',30,'Heat Crash',False,pokemon,moves)

Expanding the battle function, a shorter code can be written to try multiple moves

In [11]:
def battle_list(p1_name,p1_lvl,p2_name,p2_lvl,move,crit,pokemon,moves):
    for m in move:
        print(m,battle(p1_name,p1_lvl,p2_name,p2_lvl,m,crit,pokemon,moves))
#battle_list('Emboar',30,'Lillipup',30,['Superpower','Ember','Flamethrower'],False,pokemon,moves)

We create a function to test the effectiveness of a pokemon using a move against all other pokemon to see the average damage done. This is run using the base stats at level 50 similar to a competitive pokemon battle.

In [12]:
def move_effectiveness(p_name,move,pokemon,moves):
    dmg = [0, 0]
    ko = [0, 0]
    for index, row in pokemon.iterrows():
        d = battle(p_name,50,index,50,move,False,pokemon,moves)
        if d[0] > 100:
            d[0] = 100
            ko[0] = ko[0] + 1
        if d[1] > 100:
            d[1] = 100
            ko[1] = ko[1] + 1
        dmg = [dmg[0]+d[0],dmg[1]+d[1]]
    dmg[0] = dmg[0]/len(pokemon)
    dmg[1] = dmg[1]/len(pokemon)
    return [move,round(dmg[0],1),round(dmg[1],1),ko[0],ko[1]]
#move_effectiveness('Tepig','Ember',pokemon,moves)

Using this new function move effectiveness in combination with the list of moves a pokemon knows we can create a dataframe of the pokemon's moves and their effectiveness against all pokemon.

In [13]:
def pokemon_effectiveness(p_name,pokemon):
    name = []
    kolb = []
    koub = []
    lb = []
    ub = []
    acc = []
    typ = []
    cat = []
    pickle_in = open('all_gen5_moves.pickle','rb')
    moves = pickle.load(pickle_in)
    pokemon_moves = get_moves(p_name,moves)
    from tqdm import tnrange, notebook
    for index, row in notebook.tqdm(pokemon_moves.iterrows(),desc='main loop',total=len(pokemon_moves)):
        try:
            temp = []
            temp = move_effectiveness(p_name,index,pokemon,pokemon_moves)
            name.append(temp[0])
            lb.append(temp[1])
            ub.append(temp[2])
            kolb.append(temp[3])
            koub.append(temp[4])
            acc.append(pokemon_moves[pokemon_moves.index==index].ACCURACY.values)
            typ.append(pokemon_moves[pokemon_moves.index==index].TYPE.values)
            cat.append(pokemon_moves[pokemon_moves.index==index].CATEGORY.values)
        except ValueError as err:
            print(index,err)
        except TypeError as err:
            print(index,err)
    df = pd.DataFrame({'MOVE':name})
    df['LOWER BOUND'] = lb
    df['UPPER BOUND'] = ub
    df['GUARANTEED OHKO'] = kolb
    df['OHKO RANGE'] = koub
    df['ACCURACY'] = acc
    df['ACCURACY'] = df['ACCURACY'].astype(int)
    df['CONSISTENCY'] = df['LOWER BOUND'].astype(float)*df['ACCURACY'].astype(float)/100
    df['CONSISTENCY'] = df['CONSISTENCY'].round(1)
    df['TYPE'] = typ
    df['CATEGORY'] = cat
    df = df.set_index('MOVE')
    df = df.sort_values(by='CONSISTENCY',ascending=False)
    return df
#pokemon_effectiveness('Emboar',pokemon)